In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import os
import shutil
import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [5]:
#----------------------------------------------------------------------------
# Partimos de un conjunto de observaciones mezcladas
# y nos quedamos con los .pfds de nchan=64
#----------------------------------------------------------------------------

# creamos una lista con todos los archivos .pfd

ant = 'A2'                                                # antena usada (A1 o A2)

timing_folder = './' + str(ant) + '/'
pfd_files_total = glob.glob(timing_folder+'*pfd')         # lista de archivos .pfd
output = "output.dat"                                     # nombre del archivo de salida del cual vamos leyendo los headers

# para cada archvio .pfd, generamos un archivo con su header y leemos su nbins y su nchan

n=0
pfd_files_64 = []

for pfd in pfd_files_total:
    
    subprocess.call( "psrstat >" + output + " " + pfd , shell=True )
    n_chan = np.genfromtxt ( output, comments="none", dtype=int, skip_header=2, max_rows=1, usecols=(5) )


    if n_chan==64:                                  # si el .pfd tiene nchan=64           
        pfd_files_64.append(pfd)                    # entonces lo guardamos en la lista
        n += 1                              

In [7]:
#------------------------------------------------------------------------
# Esto es para mover las observaciones con nbins=1024 a una nueva carpeta
#------------------------------------------------------------------------

for i in range(len(pfd_files_64)):
     shutil.move(pfd_files_64[i], "./")
#    subprocess.call(["cp", pfd_files[i], "/A2_1024"])
#    shutil.copyfile(pfd_files[i], "/A2_1024"", *, follow_symlinks=True)
#    os.system( "cp " + pfd_files[i] + " /A2_1024" , shell=True )

In [16]:
#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nchan=64,32,16,8,...
#--------------------------------------------------------

ant = 'A2'                                                 # antena usada (A1 o A2)
nchan = 4                                                 # numero de bines en fase

timing_folder = './' + str(ant) + '_nchan' + str(nchan) + '/'
pfd_files = glob.glob(timing_folder+'*pfd')               # lista de archivos .pfd

#print(pfd_files)

In [17]:
#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

template = "./templates/template_1024_" + str(nchan) + "_" + str(ant) + ".std.sm"  # nombre del template
timing_file = "timing_nchan" + str(nchan) + "_" + str(ant) + ".tim"       # nombre del archivo .tim

print("template = " + str(template) )
print("timing file = " + str(timing_file) )

# primero, generamos un string que contenga todos los archivos .pfd

archivos = ''
for pfd in pfd_files:
    archivos = archivos + pfd + ' '
       
# ahora llamamos a pat para generar el archivo .tim

subprocess.call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos , shell=True )

template = ./templates/template_1024_4_A2.std.sm
timing file = timing_nchan4_A2.tim


0

In [48]:
#--------------------------------------------
# En caso de trabajar con observaciones de A1
#--------------------------------------------

subprocess.call( "sed -i -e 's/1414.8750/1414.4375/g' " + timing_file , shell=True )

0

In [7]:
#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

residue_file = "residues_" + str(nchan) + "_" + str(ant) + ".dat"   # nombre del archivo de los residuos

subprocess.call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file , shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

RMS = np.genfromtxt ( residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

print("RMS = " + str(RMS) )

RMS = 0.691
